In [1]:
import polars as pl
economics_long = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv")
print(economics_long.head())
print(economics_long.columns)

shape: (5, 5)
┌──────────┬────────────┬──────────┬───────┬──────────┐
│ rownames ┆ date       ┆ variable ┆ value ┆ value01  │
│ ---      ┆ ---        ┆ ---      ┆ ---   ┆ ---      │
│ i64      ┆ str        ┆ str      ┆ f64   ┆ f64      │
╞══════════╪════════════╪══════════╪═══════╪══════════╡
│ 1        ┆ 1967-07-01 ┆ pce      ┆ 506.7 ┆ 0.0      │
│ 2        ┆ 1967-08-01 ┆ pce      ┆ 509.8 ┆ 0.000265 │
│ 3        ┆ 1967-09-01 ┆ pce      ┆ 515.6 ┆ 0.000762 │
│ 4        ┆ 1967-10-01 ┆ pce      ┆ 512.2 ┆ 0.000471 │
│ 5        ┆ 1967-11-01 ┆ pce      ┆ 517.4 ┆ 0.000916 │
└──────────┴────────────┴──────────┴───────┴──────────┘
['rownames', 'date', 'variable', 'value', 'value01']


In [2]:
#Question 1 A

economics_wide = economics_long.pivot(
    index=["date"],
    on="variable",
    values=["value"]
)

economics_wide

date,pce,pop,psavert,uempmed,unemploy
str,f64,f64,f64,f64,f64
"""1967-07-01""",506.7,198712.0,12.6,4.5,2944.0
"""1967-08-01""",509.8,198911.0,12.6,4.7,2945.0
"""1967-09-01""",515.6,199113.0,11.9,4.6,2958.0
"""1967-10-01""",512.2,199311.0,12.9,4.9,3143.0
"""1967-11-01""",517.4,199498.0,12.8,4.7,3066.0
…,…,…,…,…,…
"""2014-12-01""",12062.0,319746.157,7.6,12.9,8717.0
"""2015-01-01""",12046.0,319928.646,7.7,13.2,8903.0
"""2015-02-01""",12082.4,320074.511,7.9,12.9,8610.0


In [3]:
#Question 1 B
median_unemploy = (
    economics_wide
    .filter(pl.col("date") >= '2010-01-01')
    .group_by("date")
    .agg(pl.col("unemploy").median().alias("median_unemployment"))
)
print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")

The median unemployment for 2010-01-01 and beyond is: shape: (64, 2)
┌────────────┬─────────────────────┐
│ date       ┆ median_unemployment │
│ ---        ┆ ---                 │
│ str        ┆ f64                 │
╞════════════╪═════════════════════╡
│ 2010-01-01 ┆ 15046.0             │
│ 2014-05-01 ┆ 9859.0              │
│ 2014-02-01 ┆ 10349.0             │
│ 2012-05-01 ┆ 12660.0             │
│ 2014-04-01 ┆ 9702.0              │
│ …          ┆ …                   │
│ 2011-07-01 ┆ 13763.0             │
│ 2015-03-01 ┆ 8504.0              │
│ 2012-04-01 ┆ 12646.0             │
│ 2013-02-01 ┆ 11950.0             │
│ 2012-07-01 ┆ 12656.0             │
└────────────┴─────────────────────┘


In [4]:
#Question 2
air_passengers = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt", separator="\t")

air_passengers.head()

Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201


In [5]:
#Question 2 A
month_cols = [col for col in air_passengers.columns if col not in ["Year"]]

air_passengers_long = air_passengers.unpivot(
    index=["Year"],
    on=month_cols,
    variable_name="Month",
).with_columns(pl.col("Month"))

air_passengers_long.head(10)

Year,Month,value
i64,str,i64
1949,"""Jan""",112
1950,"""Jan""",115
1951,"""Jan""",145
1952,"""Jan""",171
1953,"""Jan""",196
1954,"""Jan""",204
1955,"""Jan""",242
1956,"""Jan""",284
1957,"""Jan""",315


In [6]:
#Question 2 B

yearly_passengers = (
    air_passengers_long
    .filter((pl.col("Year") >= 1955) & (pl.col("Year") <= 1960))
    .group_by("Year")
    .agg(pl.col("value").sum().alias("yearly_passengers"))
)

std_passengers = (
    yearly_passengers
    .select(pl.col("yearly_passengers").std().alias("std_passengers"))
)
print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers[0, 'std_passengers']:.2f}")


The standard deviation of airline passengers between 1955 and 1960 is: 824.10


In [7]:
#Question 3
table2 = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv")
print("Original data:")
print(table2)


Original data:
shape: (12, 5)
┌──────────┬─────────────┬──────┬────────────┬────────────┐
│ rownames ┆ country     ┆ year ┆ type       ┆ count      │
│ ---      ┆ ---         ┆ ---  ┆ ---        ┆ ---        │
│ i64      ┆ str         ┆ i64  ┆ str        ┆ i64        │
╞══════════╪═════════════╪══════╪════════════╪════════════╡
│ 1        ┆ Afghanistan ┆ 1999 ┆ cases      ┆ 745        │
│ 2        ┆ Afghanistan ┆ 1999 ┆ population ┆ 19987071   │
│ 3        ┆ Afghanistan ┆ 2000 ┆ cases      ┆ 2666       │
│ 4        ┆ Afghanistan ┆ 2000 ┆ population ┆ 20595360   │
│ 5        ┆ Brazil      ┆ 1999 ┆ cases      ┆ 37737      │
│ …        ┆ …           ┆ …    ┆ …          ┆ …          │
│ 8        ┆ Brazil      ┆ 2000 ┆ population ┆ 174504898  │
│ 9        ┆ China       ┆ 1999 ┆ cases      ┆ 212258     │
│ 10       ┆ China       ┆ 1999 ┆ population ┆ 1272915272 │
│ 11       ┆ China       ┆ 2000 ┆ cases      ┆ 213766     │
│ 12       ┆ China       ┆ 2000 ┆ population ┆ 1280428583 │
└─────────

In [8]:
#Question 3 A
table2_wide = table2.pivot(
    index=["country", "year"],
    on="type",
    values="count"
)

table2_wide

country,year,cases,population
str,i64,i64,i64
"""Afghanistan""",1999,745,19987071
"""Afghanistan""",2000,2666,20595360
"""Brazil""",1999,37737,172006362
"""Brazil""",2000,80488,174504898
"""China""",1999,212258,1272915272
"""China""",2000,213766,1280428583


In [9]:
#Question 3 B
table2_with_ratio = table2_wide.with_columns(
    (pl.col("cases") / pl.col("population")).alias("ratio")
)

year_1999 = (
    table2_with_ratio
    .filter(pl.col("year") == 1999)
    .sort("ratio", descending=True)

)
highest_ratio_country = year_1999.head(1)
print("Data for 1999 with ratios:")
print(year_1999)
print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬────────┬────────────┬──────────┐
│ country     ┆ year ┆ cases  ┆ population ┆ ratio    │
│ ---         ┆ ---  ┆ ---    ┆ ---        ┆ ---      │
│ str         ┆ i64  ┆ i64    ┆ i64        ┆ f64      │
╞═════════════╪══════╪════════╪════════════╪══════════╡
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 ┆ 0.000167 │
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   ┆ 0.000037 │
└─────────────┴──────┴────────┴────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: shape: (1, 5)
┌─────────┬──────┬───────┬────────────┬──────────┐
│ country ┆ year ┆ cases ┆ population ┆ ratio    │
│ ---     ┆ ---  ┆ ---   ┆ ---        ┆ ---      │
│ str     ┆ i64  ┆ i64   ┆ i64        ┆ f64      │
╞═════════╪══════╪═══════╪════════════╪══════════╡
│ Brazil  ┆ 1999 ┆ 37737 ┆ 172006362  ┆ 0.000219 │
└─────────┴──────┴───────┴────────────┴──────────┘
